In [820]:
# 最高ウェイト表現のウェイトを最高ウェイトとの差で表す関数。
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [821]:
# Parabolic subgroupを表すクラス
class ParabolicSubgroup:
    def __init__(self, G, L, crossed_out_nodes) -> None:
        self.G = G
        self.L = L
        self.crossed_out_nodes = crossed_out_nodes
        self.R_G = WeylCharacterRing(self.G)
        self.R_L = WeylCharacterRing(self.L)

    def __repr__(self) -> str:
        return f'the parabolic subgroup of {self.G} with crossed-out nodes {self.crossed_out_nodes}'

    def dynkin_diagram(self):
        return self.G.marked_nodes(self.crossed_out_nodes).dynkin_diagram()

    def ambient_space(self):
        return self.R_G.space()
    
    def ambient_space_dimension(self):
        return self.R_G.space().dimension()

    # Sagemathのライブラリでは1-indexを用いるが、ここでは簡単のためとりあえず0-indexで表す
    def simple_roots(self):
        return [ self.R_G.simple_roots()[i] for i in set(1..self.G.rank()) - set(self.crossed_out_nodes)]

    def positive_roots(self):
        L = self.ambient_space()

        # positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
        roots = [ pr for pr in self.G.root_system().root_lattice().positive_roots()]
        for i in self.crossed_out_nodes:
            roots = [ pr for pr in roots if not (pr - (self.G.root_system().root_lattice().simple_roots())[i]).is_positive_root()]

        # ambient spaceの元に変換
        return [ L(pr) for pr in roots ]

    # 引数の`weight`は基本ウェイトを基底にして表示したもの
    def weight_muliplicities(self, weight) -> dict:
        # GとLのディンキン図の頂点のずれを補正する関数
        def correct_index(index: int) -> int:
            for i in range(len(self.crossed_out_nodes)):
                if index + i < self.crossed_out_nodes[i]:
                    return index + i
            return index + len(self.crossed_out_nodes)

        fws_L = [fw for fw in self.R_L.fundamental_weights()] # conversion from 1-index to 0-index
        weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(self.crossed_out_nodes)] 
        weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(self.L.rank()))

        fws_G = [fw for fw in self.R_G.fundamental_weights()]
        weight_for_G = sum( weight[i] * fws_G[i] for i in range(self.G.rank()))

        mul_set = root_difference_multiplicities(self.R_L, weight_for_L)

        result = {}
        for k, v in mul_set.items():
            w = weight_for_G \
                + sum(k[i - 1] * self.R_G.simple_roots()[correct_index(i)] 
                    for i in (1..self.L.rank()))
            result[w] = v

        return result
    
    def weyl_dimension(self, weight) -> int:
        return self.ambient_space().weyl_dimension(sum(weight[i - 1] * self.R_G.fundamental_weights()[i] for i in (1..self.G.rank())))

In [822]:
P = ParabolicSubgroup(CartanType('A3'), CartanType('A2'), [1])
P

the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [823]:
P.dynkin_diagram()

X---O---O
1   2   3   
A3 with node 1 marked

In [824]:
P.ambient_space_dimension()

4

In [825]:
P.simple_roots()

[(0, 1, -1, 0), (0, 0, 1, -1)]

In [826]:
P.positive_roots()

[(0, 1, -1, 0), (0, 0, 1, -1), (0, 1, 0, -1)]

In [827]:
P.weight_muliplicities((1, 2, 0))

{(3, 2, 0, 0): 1,
 (3, 1, 1, 0): 1,
 (3, 0, 2, 0): 1,
 (3, 1, 0, 1): 1,
 (3, 0, 1, 1): 1,
 (3, 0, 0, 2): 1}

In [828]:
from abc import ABC, abstractmethod
import re

class IVariety(ABC):
    @abstractmethod
    def dimension(self) -> int:
        pass

    @abstractmethod
    def tangent_bundle(self):
        pass

    @abstractmethod
    def integration(self, f) -> int:
        pass

    def chern_classes(self) -> list:
        return self.tangent_bundle().cheern_classes()

    def todd_classes(self) -> list:
        return self.tangent_bundle().todd_classes()


class IVectorBundle(ABC):
    @abstractmethod
    def base(self) -> IVariety:
        pass

    @abstractmethod
    def rank(self) -> int:
        pass

    @abstractmethod
    def chern_classes(self) -> list:
        pass

    def chern_character(self) -> list:
        # 基底空間の次元と同じであることを期待する
        len_cc = len(self.chern_classes()) - 1

        ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))
        
        # Using Singular, calclate universal formula of chern character
        singular.lib('chern.lib')
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        ch_str_list = singular.chAll(l, self.base().dimension()).sage_structured_str_list()
        chern_character = [ ring_for_ch(self.rank()) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ chern_character[i](chern_classes) for i in (0..self.base().dimension()) ]

    def todd_classes(self) -> list:
        # 基底空間の次元と同じであることを期待する
        len_cc = len(self.chern_classes()) - 1

        ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

        # Using Singular, calculate universal formula of Todd classes
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        todd_str_list = singular.todd(l).sage_structured_str_list()
        todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ todd_classes[i](chern_classes) for i in (0..len_cc) ]
        

In [829]:
# `degree`次部分を取り出す関数
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [830]:
class FlagVariety(IVariety):
    def __init__(self, parabolic_subgroup) -> None:
        self.parabolic_subgroup = parabolic_subgroup

        # コホモロジー環を含む環
        self.ring = PolynomialRing(QQ, 'x', parabolic_subgroup.ambient_space_dimension()) 
        self.x = self.ring.gens()

        self.tangent_weights = [
            sum( r[l] * self.x[l] for l in range(parabolic_subgroup.ambient_space_dimension())) 
            for r in set(parabolic_subgroup.R_G.positive_roots())-set(parabolic_subgroup.positive_roots())
        ]
        self.dim = len(self.tangent_weights)

    def __repr__(self) -> str:
        return f'a flag_variety associated to {self.parabolic_subgroup}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        tangent_weights = { w: 1 for w in set(self.parabolic_subgroup.R_G.positive_roots())-set(self.parabolic_subgroup.positive_roots()) }
        return EquivariantVectorBundle(self, tangent_weights)

    # 次数ごとのchern類
    def chern_classes(self):
        return [ 
            homogeneous_part(prod(1+x for x in self.tangent_weights), i)
            for i in (0..self.dim) 
        ]

    def numerical_integration_by_localization(self, f):
        random_x = [RealField(1000)(random()) for i in range(self.parabolic_subgroup.ambient_space_dimension())]
        orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(self.parabolic_subgroup.G)]
        top_of_f = homogeneous_part(f, self.dim)
        denominator_in_localization = prod(self.tangent_weights)
        return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(self.parabolic_subgroup.L))

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)
     

In [831]:
X = FlagVariety(P)
X

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [832]:
X.dimension()

3

In [833]:
X.tangent_weights

[x0 - x2, x0 - x1, x0 - x3]

In [834]:
X.tangent_bundle()

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to {(1, 0, -1, 0): 1, (1, -1, 0, 0): 1, (1, 0, 0, -1): 1}

In [835]:
X.chern_classes()

[1,
 3*x0 - x1 - x2 - x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 x0^3 - x0^2*x1 - x0^2*x2 + x0*x1*x2 - x0^2*x3 + x0*x1*x3 + x0*x2*x3 - x1*x2*x3]

In [836]:
X.tangent_bundle().chern_classes()

[1,
 3*x0 - x1 - x2 - x3,
 3*x0^2 - 2*x0*x1 - 2*x0*x2 + x1*x2 - 2*x0*x3 + x1*x3 + x2*x3,
 x0^3 - x0^2*x1 - x0^2*x2 + x0*x1*x2 - x0^2*x3 + x0*x1*x3 + x0*x2*x3 - x1*x2*x3]

In [837]:
X.todd_classes()

[1,
 3/2*x0 - 1/2*x1 - 1/2*x2 - 1/2*x3,
 x0^2 - 2/3*x0*x1 + 1/12*x1^2 - 2/3*x0*x2 + 1/4*x1*x2 + 1/12*x2^2 - 2/3*x0*x3 + 1/4*x1*x3 + 1/4*x2*x3 + 1/12*x3^2,
 3/8*x0^3 - 3/8*x0^2*x1 + 1/12*x0*x1^2 - 3/8*x0^2*x2 + 7/24*x0*x1*x2 - 1/24*x1^2*x2 + 1/12*x0*x2^2 - 1/24*x1*x2^2 - 3/8*x0^2*x3 + 7/24*x0*x1*x3 - 1/24*x1^2*x3 + 7/24*x0*x2*x3 - 1/8*x1*x2*x3 - 1/24*x2^2*x3 + 1/12*x0*x3^2 - 1/24*x1*x3^2 - 1/24*x2*x3^2]

In [838]:
class EquivariantVectorBundle(IVectorBundle):
    def __init__(self, flag_variety, weight_muliplicities) -> None:
        self.flag_variety = flag_variety
        self.weight_muliplicities = weight_muliplicities
        self.rk = sum(v for v in self.weight_muliplicities.values())

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight_muliplicities}'

    def rank(self) -> int:
        return self.rk

    def base(self) -> FlagVariety:
        return self.flag_variety

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        cc = prod((1 + class_from_weight(vector(w)))^i for w, i in self.weight_muliplicities.items())

        return [
            homogeneous_part(cc, i)
            for i in (0..self.flag_variety.dim)
        ]

class IrreducibleEquivariantVectorBundle(EquivariantVectorBundle):
    def __init__(self, flag_variety, weight) -> None:
        self.weight = weight
        super().__init__(flag_variety, flag_variety.parabolic_subgroup.weight_muliplicities(weight))

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight}'

In [839]:
w = (4, 0, 0)
E = IrreducibleEquivariantVectorBundle(X, w)
E

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1] associated to (4, 0, 0)

In [840]:
E.base()

a flag_variety associated to the parabolic subgroup of ['A', 3] with crossed-out nodes [1]

In [841]:
E.rank()

1

In [842]:
E.chern_classes()

[1, 4*x0, 0, 0]

In [843]:
E.chern_character()

[1, 4*x0, 8*x0^2, 32/3*x0^3]

In [844]:
E.todd_classes()

[1, 2*x0, 4/3*x0^2, 0]

In [845]:
class CompleteIntersection(IVariety):
    def __init__(self, flag_variety, vector_bundle) -> None:
        self.flag_variety = flag_variety
        self.vector_bundle = vector_bundle
        self.dim = self.flag_variety.dimension() - self.vector_bundle.rank()

    def __repr__(self) -> str:
        return f'a complete intersection of {self.flag_variety} and {self.vector_bundle}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        ci = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return ci
            def rank(self) -> int:
                return ci.dim
            def chern_classes(self) -> list:
                return ci.chern_classes()
        
        return VB()

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        def geometric_sequence(n, x):
            return sum( x^i for i in (0..n))
        
        cc = prod(1+x for x in self.flag_variety.tangent_weights) \
            * \
            prod(geometric_sequence(self.dim, -class_from_weight(vector(w)))^i 
                for w, i in self.vector_bundle.weight_muliplicities.items()
            )
        
        return [
            homogeneous_part(cc, i)
            for i in (0..self.dim)
        ]

    def numerical_integration_by_localization(self, f):
        top_of_f = homogeneous_part(f, self.dim)
        c_top    = self.vector_bundle.chern_classes()[self.vector_bundle.rank()] if self.dim >= 0 else 0
        return self.flag_variety.numerical_integration_by_localization(top_of_f * c_top)

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)


In [846]:
# K3
Y = CompleteIntersection(X, E) 
Y.dimension()

2

In [847]:
Y.chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [848]:
Y.tangent_bundle().chern_classes()

[1,
 -x0 - x1 - x2 - x3,
 7*x0^2 + 2*x0*x1 + 2*x0*x2 + x1*x2 + 2*x0*x3 + x1*x3 + x2*x3]

In [849]:
Y.integration(Y.chern_classes()[Y.dimension()])

24

In [850]:
#import re
# # `chern_classes`は0次を省いて渡す
# def hrr(integral, rank_E, chern_classes_of_E: list, chern_classes_of_M: list) -> int:
#     dim_M  = len(chern_classes_of_M)
#     len_cE = len(chern_classes_of_E)

#     # Define rings where classes lie
#     ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cE) ], order=TermOrder('wdeglex', tuple(1..len_cE)))
#     ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..dim_M) ], order=TermOrder('wdeglex', tuple(1..dim_M)))

#     # Using Singular, calclate universal formula of chern character
#     singular.lib('chern.lib')
#     r = singular.ring(0, f'(c(1..{len_cE}))', 'dp')
#     l = singular.list(f'c(1..{len_cE})')
#     ch_str_list = singular.chAll(l, dim_M).sage_structured_str_list()
#     chern_character = [ ring_for_ch(rank_E) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

#     # Using Singular, calculate universal formula of Todd classes
#     r = singular.ring(0, f'(c(1..{dim_M}))', 'dp')
#     l = singular.list(f'c(1..{dim_M})')
#     todd_str_list = singular.todd(l).sage_structured_str_list()
#     todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]
    
#     # Return the value of integration
#     return integral( 
#         sum(
#             chern_character[i](chern_classes_of_E)
#             * todd_classes[dim_M - i](chern_classes_of_M)
#         for i in (0..dim_M))
#     )

def euler_characteristic(variety: IVariety, vector_bundle: IVectorBundle) -> int:
    chern_character = vector_bundle.chern_character()
    todd_classes    = variety.todd_classes()
    
    return variety.integration( 
        sum(
            chern_character[i]
            * todd_classes[variety.dimension() - i]
            for i in (0..variety.dimension())
        )
    )



In [854]:
euler_characteristic(X, E)

35

In [855]:
P.weyl_dimension(w)

35

======= F4 cases =======

In [856]:
G = CartanType('F4')
G.dynkin_diagram()

O---O=>=O---O
1   2   3   4   
F4

In [857]:
P1 = ParabolicSubgroup(G, CartanType('A1', 'A2'), [2])
P1

the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [858]:
P1.dynkin_diagram()

O---X=>=O---O
1   2   3   4   
F4 with node 2 marked

In [859]:
X1 = FlagVariety(P1)
X1

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2]

In [860]:
w1 = (0, 1, 1, 0)
E1 = IrreducibleEquivariantVectorBundle(X1, w1)
E1

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [2] associated to (0, 1, 1, 0)

In [861]:
E1.rank()

3

In [862]:
euler_characteristic(X1, E1)

107406

In [863]:
P1.weyl_dimension(w1)

107406

In [877]:
w2 = (5, 0, 0, 0)
E2 = IrreducibleEquivariantVectorBundle(X1, w2)
E2.rank()

6

In [865]:
euler_characteristic(X1, E2)

627912

In [866]:
P1.weyl_dimension(w2)

627912

In [867]:
P2 = ParabolicSubgroup(G, CartanType('A2', 'A1'), [3])
P2

the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [868]:
P2.dynkin_diagram()

O---O=>=X---O
1   2   3   4   
F4 with node 3 marked

In [869]:
X2 = FlagVariety(P2)
X2

a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3]

In [870]:
w3 = (0, 1, 1, 0)
E3 = IrreducibleEquivariantVectorBundle(X2, w3)
E3

an equivariant vector bundle on a flag_variety associated to the parabolic subgroup of ['F', 4] with crossed-out nodes [3] associated to (0, 1, 1, 0)

In [871]:
euler_characteristic(X2, E3)

107406

In [872]:
P2.weyl_dimension(w3)

107406

In [876]:
w4 = (2, 1, 0, 0)
E4 = IrreducibleEquivariantVectorBundle(X2, w4)
E4.rank()

15

In [874]:
euler_characteristic(X2, E4)

340119

In [875]:
P2.weyl_dimension(w4)

340119